In [82]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scripts.helpers import extract_from_notation, Mode

LOCAL_DATA_PATH = "../data/raw/gun-violence-data_01-2013_03-2018.csv"
LOCAL_OUTPUT_PLOTS_PATH = "../documentation/plots/"

pd.set_option("display.max_columns", 100)
df = pd.read_csv(LOCAL_DATA_PATH)
df.shape
df.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,0::Unknown||1::Unknown,0::Unknown||1::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,NaN,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,NaN,-104.8020,NaN,NaN,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


In [84]:
# Kolumny do zachowania
columns_to_keep = [
    "n_killed", "n_injured", "n_guns_involved",
    "gun_stolen", "gun_type", "incident_characteristics",
    "participant_age_group", "participant_gender",
    "participant_status", "participant_type"
]

# Zachowujemy tylko wybrane kolumny
df = df[columns_to_keep]

# Wyświetlamy nowy kształt i podgląd danych
print("Nowy rozmiar ramki danych:", df.shape)
df.head()

Nowy rozmiar ramki danych: (239677, 10)


,n_killed,n_injured,n_guns_involved,gun_stolen,gun_type,incident_characteristics,participant_age_group,participant_gender,participant_status,participant_type
0,0,4,NaN,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...
1,1,3,NaN,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...
2,1,3,2.0,0::Unknown||1::Unknown,0::Unknown||1::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...
3,4,0,NaN,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...
4,2,2,2.0,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...


In [85]:
# Funkcje pomocnicze
def categorize_killed(n):
    if pd.isna(n):
        return "Unknown killed"
    elif n == 0:
        return "No one killed"
    elif n <= 2:
        return "Few killed"
    else:
        return "Mass killing"

def categorize_injured(n):
    if pd.isna(n):
        return "Unknown injured"
    elif n == 0:
        return "No one injured"
    elif n <= 2:
        return "Few injured"
    else:
        return "Mass injury"

def categorize_guns(n):
    if pd.isna(n):
        return "Unknown guns"
    elif n == 1:
        return "Single gun"
    elif n == 2:
        return "Two guns"
    else:
        return "Multiple guns"

# Konwersja kolumn
df["n_killed"] = df["n_killed"].apply(categorize_killed)
df["n_injured"] = df["n_injured"].apply(categorize_injured)
df["n_guns_involved"] = df["n_guns_involved"].apply(categorize_guns)

# Podgląd po konwersji
df.head()


,n_killed,n_injured,n_guns_involved,gun_stolen,gun_type,incident_characteristics,participant_age_group,participant_gender,participant_status,participant_type
0,No one killed,Mass injury,Unknown guns,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...
1,Few killed,Mass injury,Unknown guns,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...
2,Few killed,Mass injury,Two guns,0::Unknown||1::Unknown,0::Unknown||1::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...
3,Mass killing,No one injured,Unknown guns,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...
4,Few killed,Few injured,Two guns,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...


In [87]:
import pandas as pd
import numpy as np

# ---------- Pomocnicze funkcje ----------

def extract_dict_from_notation(cell):
    """Zamienia 0::Value||1::Value na słownik {0: Value, 1: Value}"""
    if pd.isna(cell):
        return {}
    result = {}
    for item in str(cell).split("||"):
        if "::" in item:
            idx, val = item.split("::", 1)
            result[int(idx)] = val
    return result

def extract_from_notation(cell):
    """Zamienia 0::Value||1::Value na listę wartości [Value, Value]"""
    if pd.isna(cell):
        return []
    return [item.split("::")[1] for item in str(cell).split("||") if "::" in item]

def safe_concat_lists(*lists):
    """Łączy bezpiecznie listy stringów, ignorując błędne typy"""
    result = []
    for l in lists:
        if isinstance(l, list):
            result += [str(x) for x in l]
        elif pd.notna(l):
            result.append(str(l))
    return result

def combine_participant_attributes(row):
    combined = []

    types = row["participant_type_dict"]
    statuses = row["participant_status_dict"]
    genders = row["participant_gender_dict"]
    ages = row["participant_age_group_dict"]

    for idx, ptype in types.items():
        if idx in statuses:
            combined.append(f"{ptype} {statuses[idx]}")
        if idx in genders:
            combined.append(f"{ptype} {genders[idx]}")
        if idx in ages:
            combined.append(f"{ptype} {ages[idx]}")

    return combined

# ---------- Wczytanie danych ----------

# df = pd.read_csv(...) # już wczytane wcześniej

# ---------- Przetwarzanie pól notacji ----------

# Kolumny notacyjne jako listy
notated_columns = ["gun_stolen", "gun_type"]
for col in notated_columns:
    df[col] = df[col].apply(extract_from_notation)

# Kolumny numeryczne na listy stringów
for col in ["n_killed", "n_injured", "n_guns_involved"]:
    df[col] = df[col].apply(lambda x: [str(x)] if pd.notna(x) else [])

# incident_characteristics rozbijamy na listę (oddzielane ||)
df["incident_characteristics"] = df["incident_characteristics"].apply(
    lambda x: str(x).split("||") if pd.notna(x) else []
)

# participant_* jako słowniki
participant_cols = ["participant_type", "participant_status", "participant_gender", "participant_age_group"]
for col in participant_cols:
    df[col + "_dict"] = df[col].apply(extract_dict_from_notation)

# Generowanie cech łączonych
df["participant_combinations"] = df.apply(combine_participant_attributes, axis=1)

# ---------- Składanie pełnej transakcji ----------

df["transaction"] = df.apply(lambda row: safe_concat_lists(
    row["n_killed"],
    row["n_injured"],
    row["n_guns_involved"],
    row["gun_stolen"],
    row["gun_type"],
    row["incident_characteristics"],
    row["participant_combinations"]
), axis=1)

# Podgląd
df["transaction"].head()


0    [No one killed, Mass injury, Unknown guns, Sho...
1    [Few killed, Mass injury, Unknown guns, Shot -...
2    [Few killed, Mass injury, Two guns, Unknown, U...
3    [Mass killing, No one injured, Unknown guns, S...
4    [Few killed, Few injured, Two guns, Unknown, U...
Name: transaction, dtype: object

In [88]:
from mlxtend.preprocessing import TransactionEncoder

transactions = df["transaction"].tolist()
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_apriori = pd.DataFrame(te_ary, columns=te.columns_)

df_apriori.head(10)


,10mm,12 gauge,16 gauge,20 gauge,22 LR,223 Rem [AR-15],25 Auto,28 gauge,30-06 Spr,30-30 Win,300 Win,308 Win,32 Auto,357 Mag,38 Spl,380 Auto,40 SW,410 gauge,44 Mag,45 Auto,7.62 [AK-47],9mm,ATF/LE Confiscation/Raid/Arrest,ATF/LE Confiscation/Raid/Arrest|Possession (gun(s) found during commission of other crimes),ATF/LE Confiscation/Raid/Arrest|Possession (gun(s) found during commission of other crimes)|Gun(s) stolen from owner,Accidental Shooting,Accidental Shooting - Death,Accidental Shooting - Injury,Accidental Shooting at a Business,Accidental Shooting|Accidental Shooting - Death,Accidental Shooting|Accidental Shooting - Death|Accidental/Negligent Discharge,Accidental Shooting|Accidental Shooting - Death|Accidental/Negligent Discharge|Drug involvement,Accidental Shooting|Accidental Shooting - Death|Accidental/Negligent Discharge|Thought gun was unloaded,Accidental Shooting|Accidental Shooting - Injury,Accidental Shooting|Accidental Shooting - Injury|Accidental/Negligent Discharge,Accidental Shooting|Accidental Shooting - Injury|Brandishing/flourishing/open carry/lost/found,Accidental Shooting|Accidental Shooting - Injury|Cleaning gun,Accidental Shooting|Accidental Shooting - Injury|Drug involvement|Possession (gun(s) found during commission of other crimes),Accidental Shooting|Accidental/Negligent Discharge,"Accidental Shooting|Accidental/Negligent Discharge|Child Involved Incident|Child picked up & fired gun|Shots fired, no action (reported, no evidence found)",Accidental Shooting|Accidental/Negligent Discharge|Officer Involved Incident,Accidental Shooting|Accidental/Negligent Discharge|Possession (gun(s) found during commission of other crimes),Accidental Shooting|Hunting accident,Accidental/Negligent Discharge,Animal shot/killed,Animal shot/killed|Defensive Use,Armed robbery with injury/death and/or evidence of DGU found,Armed robbery with injury/death and/or evidence of DGU found|Brandishing/flourishing/open carry/lost/found,Armed robbery with injury/death and/or evidence of DGU found|Brandishing/flourishing/open carry/lost/found|Defensive Use,Armed robbery with injury/death and/or evidence of DGU found|Brandishing/flourishing/open carry/lost/found|Gun(s) stolen from owner,...,"Spree Shooting (multiple victims, multiple locations)",Stolen,Stolen/Illegally owned gun{s} recovered during arrest/warrant,Subject-Suspect Adult 18+,Subject-Suspect Arrested,Subject-Suspect Child 0-11,Subject-Suspect Female,Subject-Suspect Injured,"Subject-Suspect Injured, Arrested","Subject-Suspect Injured, Unharmed","Subject-Suspect Injured, Unharmed, Arrested",Subject-Suspect Killed,"Subject-Suspect Killed, Arrested","Subject-Suspect Killed, Injured","Subject-Suspect Killed, Unharmed","Subject-Suspect Killed, Unharmed, Arrested",Subject-Suspect Male,Subject-Suspect Teen 12-17,Subject-Suspect Unharmed,"Subject-Suspect Unharmed, Arrested",Suicide - Attempt,Suicide - Attempt|Possession (gun(s) found during commission of other crimes),Suicide^,TSA Action,Terrorism Involvement,Thought gun was unloaded,Two guns,Under the influence of alcohol or drugs (only applies to the subject/suspect/perpetrator ),Unknown,Unknown guns,Unlawful purchase/sale,Victim Adult 18+,Victim Arrested,Victim Child 0-11,Victim Female,Victim Injured,"Victim Injured, Arrested","Victim Injured, Unharmed","Victim Injured, Unharmed, Arrested",Victim Killed,"Victim Killed, Arrested","Victim Killed, Injured","Victim Killed, Unharmed","Victim Killed, Unharmed, Arrested",Victim Male,"Victim Male, female",Victim Teen 12-17,Victim Unharmed,"Victim Unharmed, Arrested",Workplace shooting (disgruntled employee)
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,True,False,False,False,False,False,False

In [103]:
min_support = 0.01

col_frequencies = df_apriori.mean()

frequent_columns = col_frequencies[col_frequencies >= min_support].index.tolist()
df_apriori_reduced = df_apriori[frequent_columns]

df_apriori_reduced.columns.tolist()

selected_columns = [
    # Broń
    'Handgun', 'Rifle', 'Shotgun', 'Single gun', 'Multiple guns',

    # Sprawca
    'Subject-Suspect Male', 'Subject-Suspect Adult 18+', 'Subject-Suspect Teen 12-17',
    'Subject-Suspect Killed', 'Subject-Suspect Arrested',

    # Ofiara
    'Victim Male', 'Victim Female', 'Victim Adult 18+', 'Victim Teen 12-17', 'Victim Killed',

    # Skutki
    'Few killed', 'No one killed', 'Victim Injured', 'Few injured', 'Mass injury', 'No one injured',

    # Incydenty
    'Domestic Violence', 'Drug involvement', 'Gang involvement', 'Drive-by (car to street, car to car)',
    'Home Invasion', 'School Incident', 'Shot - Dead (murder, accidental, suicide)',
    'Shot - Wounded/Injured', 'Suicide^'
]

#df_apriori_reduced = df_apriori[selected_columns]
df_apriori_reduced = df_apriori


In [105]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(df_apriori_reduced,
                            min_support=0.05,
                            use_colnames=True,
                            verbose=1)

frequent_itemsets.sort_values(by="support", ascending=False).tail()
frequent_itemsets.shape

Processing 54 combinations | Sampling itemset size 9875


(2416, 2)

In [109]:
rules = association_rules(frequent_itemsets,
                          metric="lift",
                          min_threshold=1.25)

rules.sort_values(by="lift", ascending=False).head()
rules[rules['confidence'] > 0.6].sort_values(by="confidence", ascending=True).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
36864,"(Victim Killed, Shot - Dead (murder, accidenta...","(Victim Adult 18+, Victim Male, No one injured)",0.094527,0.164375,0.056722,0.600062,3.650557,1.0,0.041184,2.089384,0.801868,0.280552,0.521390,0.472569
43730,"(Single gun, Unknown, Few injured, Subject-Sus...","(No one killed, Victim Male, Victim Injured)",0.091360,0.271328,0.054824,0.600082,2.211651,1.0,0.030035,1.822055,0.602933,0.178078,0.451169,0.401070
1043,(Non-Shooting Incident),"(No one killed, Subject-Suspect Unharmed, Arre...",0.186756,0.206724,0.112072,0.600098,2.902895,1.0,0.073465,1.983677,0.806051,0.398253,0.495886,0.571115
37049,"(Subject-Suspect Unharmed, Arrested, Unknown, ...","(No one injured, Subject-Suspect Adult 18+, Si...",0.182308,0.186872,0.109406,0.600114,3.211361,1.0,0.075337,2.033401,0.842133,0.421156,0.508213,0.592785
18698,"(Subject-Suspect Adult 18+, Subject-Suspect Un...","(No one injured, Unknown)",0.220689,0.342978,0.132449,0.600163,1.749856,1.0,0.056758,1.643222,0.549876,0.307151,0.391440,0.493168
26408,"(No one killed, Shot - Wounded/Injured)","(Victim Adult 18+, Victim Male, Victim Injured...",0.354298,0.245218,0.212640,0.600174,2.447518,1.0,0.125760,1.887779,0.915937,0.549636,0.470277,0.733662
6786,"(Subject-Suspect Unharmed, Arrested, Subject-S...","(No one killed, Unknown)",0.242368,0.430463,0.145471,0.600207,1.394329,1.0,0.041140,1.424579,0.373280,0.275847,0.298038,0.469074
49921,"(No one killed, Subject-Suspect Adult 18+, Sho...","(Victim Adult 18+, Victim Male, Victim Injured)",0.105262,0.256316,0.063181,0.600222,2.341728,1.0,0.036200,1.860243,0.640372,0.211734,0.462436,0.423359
4856,"(Subject-Suspect Unharmed, Arrested, Subject-S...","(No one killed, No one injured)",0.242368,0.405909,0.145479,0.600241,1.478758,1.0,0.047100,1.486123,0.427327,0.289339,0.327108,0.479322
21063,"(No one killed, Victim Injured, Subject-Suspec...","(Victim Male, Subject-Suspect Adult 18+)",0.126299,0.193748,0.075810,0.600244,3.098064,1.0,0.051340,2.016862,0.775114,0.310397,0.504180,0.495764
